In [2]:
# What version of Python do you have?
import sys
import pandas as pd
import sklearn as sk
import tensorflow as tf
import tensorflow.keras as keras

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.5
Scikit-Learn 1.0.2
GPU is available


In [3]:
from keras import layers
import pyarrow.feather as feather
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# loading in the training data
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/train_16/eyesclosed_train.feather'
df_closed = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/train_16/eyesopen_train.feather'
df_open = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/train_16/mathematic_train.feather'
df_math = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/train_16/memory_train.feather'
df_memory = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/train_16/music_train.feather'
df_music = feather.read_feather(path)

In [30]:
# print shapes and data types 
print(df_closed.shape, df_open.shape, df_math.shape, df_memory.shape, df_music.shape)
print(df_closed.dtypes[0], df_open.dtypes[0], df_math.dtypes[0], df_memory.dtypes[0], df_music.dtypes[0])

(7076, 6000) (7076, 6000) (7076, 6000) (7076, 6000) (7076, 6000)
float16 float16 float16 float16 float16


In [6]:
# loading in the validation data
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/val_16/eyesclosed_val.feather'
df_closed_val = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/val_16/eyesopen_val.feather'
df_open_val = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/val_16/mathematic_val.feather'
df_math_val = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/val_16/memory_val.feather'
df_memory_val = feather.read_feather(path)
path = '/Users/asgnxt/mne-miniconda/mne_data/train_val_16/val_16/music_val.feather'
df_music_val = feather.read_feather(path)


In [7]:
# # trimming data to 1/5th duration
df_closed = df_closed.iloc[:,:int(df_closed.shape[1]/5)]
df_open = df_open.iloc[:,:int(df_open.shape[1]/5)]
df_math = df_math.iloc[:7076,:int(df_math.shape[1]/5)] #7076 rows to match other class data
df_memory = df_memory.iloc[:7076,:int(df_memory.shape[1]/5)]
df_music = df_music.iloc[:,:int(df_music.shape[1]/5)]

df_closed_val = df_closed_val.iloc[:,:int(df_closed_val.shape[1]/5)]
df_open_val = df_open_val.iloc[:,:int(df_open_val.shape[1]/5)]
df_math_val = df_math_val.iloc[:7076,:int(df_math_val.shape[1]/5)]
df_memory_val = df_memory_val.iloc[:7076,:int(df_memory_val.shape[1]/5)]
df_music_val = df_music_val.iloc[:854,:int(df_music_val.shape[1]/5)] # 854 rows to match other class data

In [12]:
df_music_val.shape

(854, 6000)

In [16]:
# defining parameters for the model

batch_size = 32

img_height = 61
img_width = 6000
target_size = (img_height, img_width)
print(f'target_size = {target_size}')

num_channels = 1
print(f'Number of channels: {num_channels}')

# calculating number of samples per class in training data
num_samples_closed = int(df_closed.shape[0]/img_height)
print(f'Number of samples in closed: {num_samples_closed}')
num_samples_open = int(df_open.shape[0]/img_height)
print(f'Number of samples in open: {num_samples_open}')
num_samples_math = int(df_math.shape[0]/img_height)
print(f'Number of samples in math: {num_samples_math}')
num_samples_memory = int(df_memory.shape[0]/img_height)
print(f'Number of samples in memory: {num_samples_memory}')
num_samples_music = int(df_music.shape[0]/img_height)
print(f'Number of samples in music: {num_samples_music}')

# calculating number of samples per class in validation data
num_samples_closed_val = int(df_closed_val.shape[0]/img_height)
print(f'Number of samples in closed_val: {num_samples_closed_val}')
num_samples_open_val = int(df_open_val.shape[0]/img_height)
print(f'Number of samples in open_val: {num_samples_open_val}')
num_samples_math_val = int(df_math_val.shape[0]/img_height)
print(f'Number of samples in math_val: {num_samples_math_val}')
num_samples_memory_val = int(df_memory_val.shape[0]/img_height)
print(f'Number of samples in memory_val: {num_samples_memory_val}')
num_samples_music_val = int(df_music_val.shape[0]/img_height)
print(f'Number of samples in music_val: {num_samples_music_val}')

# defining the number of classes
class_names = ["eyes_open", "eyes_closed", "math", "memory", "music"]
num_classes = len(class_names)
print(f'Number of classes: {num_classes}')


target_size = (61, 6000)
Number of channels: 1
Number of samples in closed: 116
Number of samples in open: 116
Number of samples in math: 116
Number of samples in memory: 116
Number of samples in music: 116
Number of samples in closed_val: 14
Number of samples in open_val: 14
Number of samples in math_val: 14
Number of samples in memory_val: 14
Number of samples in music_val: 14
Number of classes: 5


In [17]:
# converting dataframes to numpy arrays
X_closed = pd.DataFrame.to_numpy(df_closed)
X_open = pd.DataFrame.to_numpy(df_open)
X_math = pd.DataFrame.to_numpy(df_math)
X_memory = pd.DataFrame.to_numpy(df_memory)
X_music = pd.DataFrame.to_numpy(df_music)
X_closed_val = pd.DataFrame.to_numpy(df_closed_val)
X_open_val = pd.DataFrame.to_numpy(df_open_val)
X_math_val = pd.DataFrame.to_numpy(df_math_val)
X_memory_val = pd.DataFrame.to_numpy(df_memory_val)
X_music_val = pd.DataFrame.to_numpy(df_music_val)



In [18]:
# reshaping the data
X_closed = X_closed.reshape(num_samples_closed, img_height, img_width, num_channels)
X_open = X_open.reshape(num_samples_open, img_height, img_width, num_channels)
X_math = X_math.reshape(num_samples_math, img_height, img_width, num_channels)
X_memory = X_memory.reshape(num_samples_memory, img_height, img_width, num_channels)
X_music = X_music.reshape(num_samples_music, img_height, img_width, num_channels)
X_closed_val = X_closed_val.reshape(num_samples_closed_val, img_height, img_width, num_channels)
X_open_val = X_open_val.reshape(num_samples_open_val, img_height, img_width, num_channels)
X_math_val = X_math_val.reshape(num_samples_math_val, img_height, img_width, num_channels)
X_memory_val = X_memory_val.reshape(num_samples_memory_val, img_height, img_width, num_channels)
X_music_val = X_music_val.reshape(num_samples_music_val, img_height, img_width, num_channels)


In [19]:
# stacking the data
X_tr = np.vstack((X_closed, X_open, X_math, X_memory, X_music))
X_val = np.vstack((X_closed_val, X_open_val, X_math_val, X_memory_val, X_music_val))

In [26]:
print(f'training data dimensions: {X_tr.shape}')
print(f'validation data dimensions: {X_val.shape}')

training data dimensions: (580, 61, 6000, 1)
validation data dimensions: (70, 61, 6000, 1)


In [20]:
# labels for the data
labels_closed = np.zeros(num_samples_closed)
labels_open = np.ones(num_samples_open)
labels_math = np.full(num_samples_math, 2)
labels_memory = np.full(num_samples_memory, 3)
labels_music = np.full(num_samples_music, 4)
labels_closed_val = np.zeros(num_samples_closed_val)
labels_open_val = np.ones(num_samples_open_val)
labels_math_val = np.full(num_samples_math_val, 2)
labels_memory_val = np.full(num_samples_memory_val, 3)
labels_music_val = np.full(num_samples_music_val, 4)



In [21]:
# stacking the labels
y_tr = np.hstack((labels_closed, labels_open, labels_math, labels_memory, labels_music))
y_val = np.hstack((labels_closed_val, labels_open_val, labels_math_val, labels_memory_val, labels_music_val))

In [22]:
# convert class vectors to binary class matrices
y_tr = keras.utils.to_categorical(y_tr, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [27]:
print(f'training labels dimensions: {y_tr.shape}')
print(f'validation labels dimensions: {y_val.shape}')

training labels dimensions: (580, 5)
validation labels dimensions: (70, 5)


In [23]:
input_shape = (61, 6000, 1)

In [28]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_7 (ModuleWrap (None, 59, 5998, 32)      320       
_________________________________________________________________
module_wrapper_8 (ModuleWrap (None, 29, 2999, 32)      0         
_________________________________________________________________
module_wrapper_9 (ModuleWrap (None, 27, 2997, 64)      18496     
_________________________________________________________________
module_wrapper_10 (ModuleWra (None, 13, 1498, 64)      0         
_________________________________________________________________
module_wrapper_11 (ModuleWra (None, 1246336)           0         
_________________________________________________________________
module_wrapper_12 (ModuleWra (None, 1246336)           0         
_________________________________________________________________
module_wrapper_13 (ModuleWra (None, 5)                

In [29]:
# train the model
epochs = 19

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_tr, y_tr, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=1)

2022-03-27 08:36:54.920280: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-27 08:36:54.922943: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-27 08:36:55.050352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/19
19/19 [==============================] - ETA: 0s - loss: 1.6212 - accuracy: 0.1983

2022-03-27 08:37:18.086362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 24s 1s/step - loss: 1.6212 - accuracy: 0.1983 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 2/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 3/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 4/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 5/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 6/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 7/19
19/19 [==============================] - 21s 1s/step - loss: 1.6095 - accuracy: 0.2000 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 8/19
19/19 [=============